<!--
Copyright (c) 2025 Milin Patel
Hochschule Kempten - University of Applied Sciences

Autonomous Driving: AI Safety and Security Workshop
This project is licensed under the MIT License.
-->

*Copyright © 2025 Milin Patel. All Rights Reserved.*

# Notebook 20: Explainability (XAI) for Safety Certification

**Session 5: Standards Integration and Deployment**

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/milinpatel07/Autonomous-Driving_AI-Safety-and-Security/blob/master/08_Advanced_Topics/notebooks/20_Explainability_XAI.ipynb)

**Author:** Milin Patel 
**Institution:** Hochschule Kempten

---

## Learning Objectives

- Understand why explainability is mandatory for AV certification
- Learn XAI methods: LIME, SHAP, GradCAM, attention visualization
- Apply saliency maps to perception models
- Document AI decisions for regulators
- Understand XAI requirements in ISO/IEC TR 5469
- Navigate accuracy vs interpretability trade-offs

---

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
from IPython.display import Image, display
import warnings
warnings.filterwarnings('ignore')

print("✓ Libraries loaded")

---

## 1. Why Explainability Matters for AV Certification

### The Problem

Deep neural networks are **black boxes**:
- 100 million+ parameters in perception models
- Non-linear transformations across layers
- Impossible to trace decision path manually

**Regulators ask:** "Why did the vehicle brake here?"
**Traditional answer:** "The neural network output was 0.87"
**Regulator response:** "That's not an explanation. Rejected."

### Why This Matters

1. **Legal Liability**: When crashes occur, courts need to understand why AI made decisions
2. **Certification**: Regulators (NHTSA, UNECE) require explainable AI per ISO/IEC TR 5469
3. **Debugging**: Engineers need to fix failures, not just detect them
4. **Trust**: Public acceptance requires understanding
5. **Standards Compliance**: ISO 26262 requires failure analysis - can't analyze what you can't explain

### Real Example: Uber ATG Crash (2018)

- Perception system detected pedestrian 6 seconds before impact
- System classified object as: vehicle → bicycle → unknown → pedestrian
- **Critical question**: Why did classification keep changing?
- **Without XAI**: Cannot answer, cannot fix
- **With XAI**: Could visualize which image features caused confusion

### Regulatory Requirements

**ISO/IEC TR 5469 (AI Functional Safety):**
- Section 6.4: AI systems must provide "rationale for decisions"
- Section 7.2: "Interpretability shall be documented"

**EU AI Act (2024):**
- High-risk AI (includes AVs) must provide explanations
- Right to explanation for impacted individuals

**NHTSA Standing General Order:**
- Crash reports must explain AV decisions before crash

**Key Insight:** XAI is not optional - it's legally required.

---

## 2. XAI Methods for Perception

### Method 1: Saliency Maps (Gradient-Based)

**Question:** Which pixels influenced the decision most?

**Method:**
1. Forward pass: input image → prediction
2. Compute gradient: ∂(prediction)/∂(input pixels)
3. Visualize: bright pixels = high influence

**Interpretation:** Highlights regions model "looks at"

**Limitation:** Shows correlation, not causation

In [ ]:
# Example: Simple saliency map computation
def compute_saliency_map(model, image, target_class):
    """
    Compute saliency map showing which pixels influence prediction.
    
    Args:
        model: Trained neural network
        image: Input image tensor (requires_grad=True)
        target_class: Class index to explain
    
    Returns:
        Saliency map (same size as image)
    """
    model.eval()
    image.requires_grad = True
    
    # Forward pass
    output = model(image)
    
    # Get score for target class
    score = output[0, target_class]
    
    # Backward pass
    score.backward()
    
    # Gradient magnitude
    saliency = image.grad.abs().max(dim=1)[0]
    
    return saliency.detach().cpu().numpy()

print("✓ Saliency map function defined")
print("  Usage: Shows which pixels the model focuses on")

### Method 2: GradCAM (Gradient-weighted Class Activation Mapping)

**Improvement over saliency:** Shows **where** in feature maps model looks

**How it works:**
1. Get gradients flowing into last convolutional layer
2. Weight each feature map by gradient importance
3. Sum weighted feature maps
4. Overlay heatmap on original image

**Advantage:** 
- Higher resolution than simple saliency
- Shows object-level attention
- Works for any CNN architecture

**Real-world use:** 
- Waymo uses GradCAM to debug perception failures
- Shows exactly which part of object was detected

In [ ]:
# GradCAM implementation (simplified)
class GradCAM:
    """
    Gradient-weighted Class Activation Mapping for CNN explainability.
    
    Shows which regions of the image the model focuses on for a specific prediction.
    """
    
    def __init__(self, model, target_layer):
        self.model = model
        self.target_layer = target_layer
        self.gradients = None
        self.activations = None
        
        # Register hooks
        target_layer.register_forward_hook(self.save_activation)
        target_layer.register_full_backward_hook(self.save_gradient)
    
    def save_activation(self, module, input, output):
        self.activations = output.detach()
    
    def save_gradient(self, module, grad_input, grad_output):
        self.gradients = grad_output[0].detach()
    
    def generate_cam(self, image, target_class):
        """
        Generate Class Activation Map.
        
        Returns:
            Heatmap showing important regions (H x W)
        """
        # Forward pass
        output = self.model(image)
        
        # Backward pass for target class
        self.model.zero_grad()
        output[0, target_class].backward()
        
        # Weight activations by gradients
        weights = self.gradients.mean(dim=[2, 3], keepdim=True)
        cam = (weights * self.activations).sum(dim=1, keepdim=True)
        
        # ReLU and normalize
        cam = torch.relu(cam)
        cam = cam / (cam.max() + 1e-8)
        
        return cam[0, 0].cpu().numpy()

print("✓ GradCAM class defined")
print("  Usage: Shows object-level attention in perception models")

### Method 3: LIME (Local Interpretable Model-Agnostic Explanations)

**Idea:** Explain complex model by fitting simple model locally

**Algorithm:**
1. Perturb input (e.g., mask image regions)
2. Get predictions on perturbed inputs
3. Fit linear model: perturbations → predictions
4. Linear weights = feature importance

**Advantage:**
- Works for any model (model-agnostic)
- Provides local explanations
- Human-interpretable features

**Disadvantage:**
- Computationally expensive (many forward passes)
- Explanations can be unstable

**AV Application:** 
- Explain why pedestrian was missed
- Test: if we mask this region, does detection fail?

### Method 4: SHAP (SHapley Additive exPlanations)

**Based on:** Game theory (Shapley values)

**Idea:** How much does each feature contribute to prediction?

**Properties:**
- **Additivity:** Individual contributions sum to final prediction
- **Consistency:** If feature becomes more important, SHAP value increases
- **Uniqueness:** Only one valid explanation

**AV Application:**
- Decompose detection score: 0.87 = 0.3 (shape) + 0.4 (color) + 0.17 (context)
- Regulators accept because mathematically principled

**Limitation:** Exponentially expensive (need approximations)

### Method 5: Attention Visualization (for Transformers)

Modern perception uses Vision Transformers (ViT).

**Advantage:** Attention weights are built-in explanations!

**How:**
- Extract attention weights from each head
- Visualize which image patches attend to which
- See what model "looks at"

**Example:** Tesla FSD Beta uses attention visualization internally

---

## 3. Practical: Explaining Pedestrian Detection

### Scenario

Your AV perception system detects a pedestrian with confidence 0.91.
**Regulator asks:** "Why this detection?"

### Step-by-step XAI Analysis

In [ ]:
# Simulate detection explanation
fig, axes = plt.subplots(1, 4, figsize=(16, 4))

# Original image (simulated)
axes[0].set_title('1. Original Image', fontweight='bold')
axes[0].text(0.5, 0.5, 'Pedestrian\nCrossing', ha='center', va='center', 
            fontsize=14, bbox=dict(boxstyle='round', facecolor='lightblue'))
axes[0].set_xlim(0, 1)
axes[0].set_ylim(0, 1)
axes[0].axis('off')

# Saliency map
axes[1].set_title('2. Saliency Map\n(Gradient)', fontweight='bold')
saliency = np.random.rand(10, 10)
saliency[3:7, 4:6] = 0.9  # Bright region on pedestrian
axes[1].imshow(saliency, cmap='hot', interpolation='bilinear')
axes[1].text(5, 9, 'High gradient\non person shape', ha='center', fontsize=9)
axes[1].axis('off')

# GradCAM heatmap
axes[2].set_title('3. GradCAM Heatmap\n(Feature importance)', fontweight='bold')
gradcam = np.zeros((10, 10))
gradcam[3:7, 4:6] = 1.0  # Focus on pedestrian
axes[2].imshow(gradcam, cmap='jet', interpolation='bilinear', alpha=0.7)
axes[2].text(5, 9, 'Model focuses\non upper body', ha='center', fontsize=9)
axes[2].axis('off')

# SHAP explanation
axes[3].set_title('4. SHAP Values\n(Feature contributions)', fontweight='bold')
features = ['Human\nShape', 'Vertical\nPose', 'Legs\nMoving', 'Context\n(Crosswalk)', 'Background']
shap_values = [0.35, 0.28, 0.18, 0.10, 0.00]
colors = ['red' if v > 0.2 else 'orange' if v > 0.1 else 'gray' for v in shap_values]
axes[3].barh(features, shap_values, color=colors)
axes[3].set_xlabel('Contribution to\nPedestrian Score (0.91)', fontsize=9)
axes[3].set_xlim(0, 0.4)

plt.tight_layout()
plt.show()

print("\n📊 XAI Explanation for Regulators:")
print("   Detection confidence: 0.91")
print("   Primary factors:")
print("   - Human shape match: +0.35")
print("   - Vertical pose: +0.28")
print("   - Leg movement detected: +0.18")
print("   - Located in crosswalk: +0.10")
print("   Total: 0.91 ✓")
print("\n   This explanation satisfies ISO/IEC TR 5469 requirements.")

---

## 4. Documenting AI Decisions for Certification

### Required Documentation (ISO/IEC TR 5469)

For each safety-critical AI component, document:

1. **Model Architecture**
 - Layers, parameters, activations
 - Why this architecture chosen

2. **Training Data**
 - Dataset composition
 - Bias analysis
 - ODD coverage

3. **Decision Rationale**
 - XAI method used (GradCAM, SHAP, etc.)
 - Example explanations for test cases
 - Edge case analysis

4. **Failure Modes**
 - Known failure scenarios
 - Why failures occur (with XAI)
 - Mitigation strategies

5. **Validation Evidence**
 - Test coverage
 - Performance metrics
 - Uncertainty quantification

### Example Documentation Template

In [ ]:
# Example: AI Decision Documentation Template
documentation_template = """
════════════════════════════════════════════════════════════════
AI DECISION DOCUMENTATION (ISO/IEC TR 5469 Compliant)
════════════════════════════════════════════════════════════════

SYSTEM: Pedestrian Detection Module
VERSION: 2.3.1
ASIL LEVEL: ASIL-D (ISO 26262)
DATE: 2025-11-23
AUTHOR: Milin Patel, Hochschule Kempten

────────────────────────────────────────────────────────────────
1. MODEL ARCHITECTURE
────────────────────────────────────────────────────────────────
Base: YOLOv8-medium
Parameters: 25.9M
Input: 640x640 RGB
Output: Bounding boxes + class probabilities
Inference time: 23ms (average)

Rationale: YOLOv8 chosen for real-time performance (<50ms) 
required by ASIL-D latency budget.

────────────────────────────────────────────────────────────────
2. TRAINING DATA
────────────────────────────────────────────────────────────────
Dataset: nuScenes (1000 scenes) + KITTI (7481 images) + 
         Internal dataset (50K images)
Pedestrian instances: 187,342
ODD coverage: Urban (60%), Highway (25%), Residential (15%)
Weather: Clear (70%), Rain (20%), Night (10%)
Bias analysis: Gender balance 52%/48%, Age distribution verified

────────────────────────────────────────────────────────────────
3. EXPLAINABILITY METHOD
────────────────────────────────────────────────────────────────
Primary: GradCAM (on backbone layer 4)
Secondary: SHAP for feature importance
Validation: Attention rollout for transformer heads

Example Explanation (Test Case TC-PED-042):
  Input: Pedestrian in crosswalk, daylight
  Output: Confidence 0.94
  
  GradCAM Analysis:
  - High activation on upper body (torso, head)
  - Moderate activation on legs
  - Low activation on background
  
  SHAP Decomposition:
  - Body shape:  +0.38
  - Upright pose: +0.29
  - Motion:      +0.19
  - Context:     +0.08
  Total:         0.94 ✓

────────────────────────────────────────────────────────────────
4. KNOWN FAILURE MODES
────────────────────────────────────────────────────────────────
Failure FM-PED-01: Partial occlusion (>70% occluded)
  Explanation: Insufficient visible pixels for shape matching
  XAI Evidence: GradCAM shows no activation on occluded region
  Mitigation: Multi-frame temporal integration
  
Failure FM-PED-02: Extreme lighting (direct sunlight)
  Explanation: Overexposure saturates pixels, destroys edges
  XAI Evidence: Saliency map shows uniform gradients
  Mitigation: HDR image processing, sensor fusion with LiDAR

────────────────────────────────────────────────────────────────
5. CERTIFICATION EVIDENCE
────────────────────────────────────────────────────────────────
Test Coverage: 98.7% of ODD scenarios
Precision: 0.947 (on validation set)
Recall: 0.923 (SOTIF target: >0.95)
False Negative Rate: 0.077 → ASIL-D requires <0.01
  ⚠ ISSUE: Requires additional validation

Uncertainty Quantification: MC Dropout (50 samples)
  Mean epistemic uncertainty: 0.08
  High uncertainty cases manually reviewed

════════════════════════════════════════════════════════════════
CERTIFICATION STATUS: Pending - FNR improvement required
REVIEWER: [Safety Engineer Name]
APPROVAL: [Regulatory Authority]
════════════════════════════════════════════════════════════════
"""

print(documentation_template)
print("\n✓ This template satisfies ISO/IEC TR 5469 Section 7.2")
print("  'Interpretability shall be documented'")

---

## 5. Trade-offs: Accuracy vs Interpretability

### The Dilemma

**Simple models** (decision trees, linear regression):
- Easy to explain
- Lower accuracy
- May not meet safety requirements

**Complex models** (deep neural networks):
- High accuracy
- Hard to explain
- Regulators uncomfortable

### Industry Approaches

**Option 1: Post-hoc explanation (most common)**
- Use complex model for performance
- Apply XAI (SHAP, GradCAM) for explanation
- Document explanations for certification
- **Used by:** Waymo, Cruise, Tesla

**Option 2: Inherently interpretable models**
- Use attention mechanisms (Transformers)
- Attention weights = explanation
- Trade some accuracy for interpretability
- **Used by:** Some research groups

**Option 3: Hybrid approach**
- Complex model for difficult cases
- Simple model for easy cases
- Route based on scenario difficulty
- **Emerging approach**

### Certification Reality

Regulators accept complex models **if**:
1. XAI methods are validated
2. Explanations are consistent
3. Failure modes are understood
4. Extensive testing is done
5. Human oversight exists

**Key Insight:** You don't need a simple model, you need good explanations of your complex model.

---

## 6. Key Takeaways

1. **XAI is mandatory** for AV certification per ISO/IEC TR 5469 and EU AI Act

2. **Multiple XAI methods exist**: GradCAM, SHAP, LIME, saliency maps, attention

3. **No single best method**: Use multiple for robustness

4. **Documentation is critical**: Regulators need written explanations, not just visualizations

5. **Post-hoc explanation works**: Don't sacrifice accuracy for interpretability

6. **Explainability aids debugging**: Helps engineers fix failures

7. **Trust requires transparency**: Public acceptance needs understandable AI

---

## Further Reading

- Selvaraju et al. (2017): "Grad-CAM: Visual Explanations from Deep Networks"
- Lundberg & Lee (2017): "A Unified Approach to Interpreting Model Predictions" (SHAP)
- Ribeiro et al. (2016): "Why Should I Trust You? Explaining Predictions" (LIME)
- ISO/IEC TR 5469:2024: "Artificial intelligence — Functional safety and AI systems"
- EU AI Act (2024): Requirements for high-risk AI systems

---

*Copyright © 2025 Milin Patel. All Rights Reserved.*